In [1]:
from pathlib import Path
import re
from shutil import copy

In [2]:
src = "D:/Google Drive/Azna\Docs/Κτηματολόγιο Κιλκίς/Γιαγιά"
dst = "D:/.temp/copy_tests"

pattern = "<ota@3:7><tomeas@8:9><enotita@10:11><gt@12:14>"

In [3]:
class PatternMatcher:
    def __init__(self, pattern) -> None:
        self.pattern = pattern
        self.regex = re.compile('<(.*?)>')
        self.parts = self.regex.findall(self.pattern)
        self.variables = {}
        self._create_variables()

    def _create_variables(self):
        for part in self.parts:
            var_name, idxs = part.split('@')
            start_idx, end_idx = idxs.split(':')

            if start_idx:
                _start = int(start_idx) - 1
            else:
                _start = 0

            if end_idx:
                _end = int(end_idx)
            else:
                _end = None

            self.variables[var_name] = {'start': _start,
                                        'end': _end}

    def match(self, text):
        values = {}
        for var in self.variables:
            _var = f"<{var}>"
            s = self.variables[var]['start']
            e = self.variables[var]['end']
            if e is None:
                values[var] = text[s:]
            else:
                values[var] = text[s:e]

        return values


In [4]:
def replace_all(text, dic):
    for i, j in dic.items():
        text = text.replace(f"<{i}>", j)
    return text

In [5]:
pm = PatternMatcher(pattern=pattern)

In [15]:
keep = "Diagrammata/<ota>/<tomeas>/<enotita>"

In [16]:
class PatternCopy:
    def __init__(self, src:str, dst:str, pattern:PatternMatcher, create_like:str) -> None:
        self.src = Path(src)
        self.dst = Path(dst)
        self.pattern = pattern
        self.create_like = create_like

    def copy_files(self, match=str):
        for p in Path(src).glob(match):
            parts = self.pattern.match(p.stem)
            _dst = self.dst.joinpath(replace_all(self.create_like, parts))
            _dst.mkdir(parents=True, exist_ok=True)
            copy(p, _dst)

In [17]:
pc = PatternCopy(src, dst, pm, keep)

In [18]:
pc.copy_files('*.pdf')

In [ ]:
pdst = Path(dst)
for p in Path(src).glob('K*.pdf'):
    parts = pm.match(p.stem)
    go_to = pdst.joinpath(replace_all(keep, parts))
    go_to.mkdir(parents=True, exist_ok=True)
    copy(p, go_to)
    

In [ ]:
parts = pm.match("KD260391009004_0_0")
parts

In [ ]:
replace_all(keep, parts)

In [ ]:
rep = {'<ota>': '26039', '<tomeas>': '10', '<enotita>': '09', '<gt>': '004'}

rep = dict((re.escape(k), v) for k, v in rep.items()) 

pattern = re.compile("|".join(rep.keys()))
text = pattern.sub(lambda m: rep[re.escape(m.group(0))], keep)

In [ ]:
text

In [ ]:
pattern